In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))


def po_linear_model(graph, alpha=1, beta=1, sigma=0.1, gamma=2):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i]  

def po_multiplicative_model(graph, alpha=1, sigma=0.1, delta=1, gamma=2): 
    for i in graph.nodes:
        graph.nodes[i]["y"] = ( (alpha + sigma * np.random.normal()) * graph.degree[i]/avg_deg )  *  (1 + delta * graph.nodes[i]["z"] + gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]]) / len(graph[i]) )




DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["n_cl"] = len(node_to_connected_clusters[i])
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)

    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [7]:
# cluster-level cr

GATE_hat_list = np.zeros((repeat_num, len(ramps)))
     
for seed in tqdm(range(repeat_num)):
    np.random.seed(seed)     
        
    for num_step in range(len(ramps)): 
        print(f"Seed: {seed}")        
        # no ramp
        rollout_index = np.random.uniform(0, 1, size=(num_cluster,))           
        torch.manual_seed(2)
        p_list = ramps[num_step:]                
        g_feat_list = []
        y_list = []
        
        for p in p_list:            
            tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]            
            tr_units = reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters])  
                        
            nx.set_node_attributes(g, 0, "z")
            nx.set_node_attributes(g, {unit:1 for unit in tr_units}, "z")
            
            po_linear_model(g, gamma = 1)            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()  
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
            
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/NoRamp_CR_cluster_res10_final.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.02, Loss: 8.0868
Epoch 0, Treat Prop: 0.05, Loss: 2.4003
Epoch 0, Treat Prop: 0.10, Loss: 1.2841
Epoch 0, Treat Prop: 0.25, Loss: 1.8350
Epoch 0, Treat Prop: 0.50, Loss: 3.8925
Epoch 50, Treat Prop: 0.02, Loss: 0.1293
Epoch 50, Treat Prop: 0.05, Loss: 0.1905
Epoch 50, Treat Prop: 0.10, Loss: 0.1512
Epoch 50, Treat Prop: 0.25, Loss: 0.2451
Epoch 50, Treat Prop: 0.50, Loss: 0.8272
Epoch 100, Treat Prop: 0.02, Loss: 0.0305
Epoch 100, Treat Prop: 0.05, Loss: 0.0227
Epoch 100, Treat Prop: 0.10, Loss: 0.0318
Epoch 100, Treat Prop: 0.25, Loss: 0.0486
Epoch 100, Treat Prop: 0.50, Loss: 0.0674
Epoch 150, Treat Prop: 0.02, Loss: 0.0423
Epoch 150, Treat Prop: 0.05, Loss: 0.0221
Epoch 150, Treat Prop: 0.10, Loss: 0.0314
Epoch 150, Treat Prop: 0.25, Loss: 0.0528
Epoch 150, Treat Prop: 0.50, Loss: 0.0475
Epoch 200, Treat Prop: 0.02, Loss: 0.0421
Epoch 200, Treat Prop: 0.05, Loss: 0.0306
Epoch 200, Treat Prop: 0.10, Loss: 0.0260
Epoch 200, Treat Prop: 0.25, Loss: 0.0592
Epoch 2

  0%|          | 1/1000 [00:31<8:51:22, 31.91s/it]

...

 26%|██▌       | 256/1000 [2:07:12<6:04:13, 29.37s/it]

0.03771045058965683
Seed: 256
Epoch 0, Treat Prop: 0.02, Loss: 7.9947
Epoch 0, Treat Prop: 0.05, Loss: 2.9135
Epoch 0, Treat Prop: 0.10, Loss: 1.9239
Epoch 0, Treat Prop: 0.25, Loss: 2.3980
Epoch 0, Treat Prop: 0.50, Loss: 3.6855
Epoch 50, Treat Prop: 0.02, Loss: 0.1694
Epoch 50, Treat Prop: 0.05, Loss: 0.2276
Epoch 50, Treat Prop: 0.10, Loss: 0.1535
Epoch 50, Treat Prop: 0.25, Loss: 0.2433
Epoch 50, Treat Prop: 0.50, Loss: 0.4232
Epoch 100, Treat Prop: 0.02, Loss: 0.0186
Epoch 100, Treat Prop: 0.05, Loss: 0.0756
Epoch 100, Treat Prop: 0.10, Loss: 0.0362
Epoch 100, Treat Prop: 0.25, Loss: 0.0657
Epoch 100, Treat Prop: 0.50, Loss: 0.0767
Epoch 150, Treat Prop: 0.02, Loss: 0.0375
Epoch 150, Treat Prop: 0.05, Loss: 0.0384
Epoch 150, Treat Prop: 0.10, Loss: 0.0686
Epoch 150, Treat Prop: 0.25, Loss: 0.0389
Epoch 150, Treat Prop: 0.50, Loss: 0.0846
Epoch 200, Treat Prop: 0.02, Loss: 0.0274
Epoch 200, Treat Prop: 0.05, Loss: 0.0301
Epoch 200, Treat Prop: 0.10, Loss: 0.0489
Epoch 200, Treat Pr

 26%|██▌       | 257/1000 [2:07:41<6:03:12, 29.33s/it]

0.037943486124277115
Seed: 257
Epoch 0, Treat Prop: 0.02, Loss: 8.0394
Epoch 0, Treat Prop: 0.05, Loss: 2.3702
Epoch 0, Treat Prop: 0.10, Loss: 1.2321
Epoch 0, Treat Prop: 0.25, Loss: 1.9064
Epoch 0, Treat Prop: 0.50, Loss: 3.3307
Epoch 50, Treat Prop: 0.02, Loss: 0.1064
Epoch 50, Treat Prop: 0.05, Loss: 0.1525
Epoch 50, Treat Prop: 0.10, Loss: 0.1228
Epoch 50, Treat Prop: 0.25, Loss: 0.2392
Epoch 50, Treat Prop: 0.50, Loss: 0.6193
Epoch 100, Treat Prop: 0.02, Loss: 0.0222
Epoch 100, Treat Prop: 0.05, Loss: 0.0180
Epoch 100, Treat Prop: 0.10, Loss: 0.0241
Epoch 100, Treat Prop: 0.25, Loss: 0.0345
Epoch 100, Treat Prop: 0.50, Loss: 0.0512
Epoch 150, Treat Prop: 0.02, Loss: 0.0348
Epoch 150, Treat Prop: 0.05, Loss: 0.0245
Epoch 150, Treat Prop: 0.10, Loss: 0.0225
Epoch 150, Treat Prop: 0.25, Loss: 0.0498
Epoch 150, Treat Prop: 0.50, Loss: 0.0455
Epoch 200, Treat Prop: 0.02, Loss: 0.0241
Epoch 200, Treat Prop: 0.05, Loss: 0.0222
Epoch 200, Treat Prop: 0.10, Loss: 0.0195
Epoch 200, Treat P

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

